In [ ]:
import os
import pandas as pd
from poseidon.util import general
from poseidon.util.sf_client import Salesforce

pd.options.display.max_columns = 999

In [ ]:
username = "maksimp@sandiego.gov"
password = "KrakatoaIsTheBestCoffeeShopEver2"
security_token = "qn4CD0BK85EN8ZCXig2npOmN"
report_id = "00Ot0000000mCVoEAM"
client_id = 'DataSD_Poseidon'
sf_version = '29.0'
domain = 'sdgov.my'

sf = Salesforce(username, password, security_token)

df = sf.get_report_df(report_id)


df_bk = df
df.head()

[2016-12-28 22:58:51,234] {connectionpool.py:807} INFO - Starting new HTTPS connection (1): sdgov.my.salesforce.com
[2016-12-28 22:58:52,265] {connectionpool.py:807} INFO - Starting new HTTPS connection (1): sdgov.my.salesforce.com


In [ ]:
df.dtypes

In [ ]:
tests = df[(df['Public Description'].str.match('test',case=False)) | 
                      (df['Public Description'].str.contains('just a test',na=False,case=False)) | 
                      (df['Public Description'].str.contains('test only',na=False,case=False)) | 
                      (df['Public Description'].str.contains('test-',na=False,case=False)) | 
                      (df['Public Description'].str.contains('test -',na=False,case=False)) | 
                      (df['Public Description'].str.contains('test report',na=False,case=False)) | 
                      (df['Public Description'].str.contains('test case',na=False,case=False)) | 
                      (df['Public Description'].str.contains('a test',na=False,case=False)) | 
                      (df['Public Description'].str.contains('test test',na=False,case=False)) |
                      (df['Public Description'].str.contains('testing something',na=False,case=False))]

tests_series = tests['Case Number']
tests_series = tests_series.tolist()
tests_series.append(80507)
tests_series.append(104333)

final_reports = df[~df['Case Number'].isin(tests_series)]
final_reports = final_reports.reset_index(drop=True)
final_reports['Date/Time Opened'] = pd.to_datetime(final_reports['Date/Time Opened'], errors='coerce', infer_datetime_format=True)
final_reports['Date/Time Closed'] = pd.to_datetime(final_reports['Date/Time Closed'], errors='coerce', infer_datetime_format=True)
final_reports['Case Date/Time Last Modified'] = pd.to_datetime(final_reports['Case Date/Time Last Modified'], errors='coerce', infer_datetime_format=True)

final_reports.dtypes


In [ ]:
# Remove Useless Columns
final_reports.drop(['Public Description','Referred Email List','Problem Description'], axis=1, inplace=True)


In [ ]:
final_reports = final_reports.sort_values(by='Case Number', ascending=False)

final_reports = final_reports.rename(columns={'Case Number':'case_number', 
                                              'Parent Case Number':'parent_case_number', 
                                              'Status':'status',
                                              'Mobile/Web Status':'mobile_web_status', 
                                              'Duplicate Verified':'duplicate_verified', 
                                              'Override Duplicate':'override_duplicate',
                                              'Functional Location':'functional_location', 
                                              'Question #1':'question_1', 
                                              'Answer #1':'answer_1', 
                                              'Answer #1 Options':'answer_1_options',
                                              'Question #2':'question_2', 
                                              'Answer #2':'answer_2', 
                                              'Answer #2 Options':'answer_2_options', 
                                              'Question #3':'question_3',
                                              'Answer #3':'answer_3', 
                                              'Question #4':'question_4', 
                                              'Answer #4':'answer_4', 
                                              'Referral Email List':'referral_email',
                                              'Referred Email Update':'referred_email_update', 
                                              'Case Origin':'case_origin',
                                              'Date/Time Opened':'opened_date', 
                                              'Date/Time Closed':'closed_date', 
                                              'Case Date/Time Last Modified': 'last_modified_date',
                                              'Open':'open', 
                                              'Closed':'closed', 
                                              'Subject':'subject', 
                                              'Problem Group':'problem_group', 
                                              'Problem Category':'problem_category',
                                              'Problem Category Detail':'problem_category_detail', 
                                              'Geolocation Lat Long':'coordinates', 
                                              'Geolocation (Latitude)':'lat',
                                              'Geolocation (Longitude)':'lng', 
                                              'SAP Notification Number':'sap_notification_number',
                                              'SAP Problem Category':'sap_problem_category', 
                                              'SAP Problem Code':'sap_problem_code', 
                                              'SAP Problem Type':'sap_problem_type',
                                              'Street Address':'address', 
                                              'Case Record Type':'case_record_type'})

final_reports.head()

In [ ]:
final_reports.dtypes

In [ ]:
# Output final reports here.
#%Y-%m-%dT%H:%M:%S%z

final_reports['address'].replace('\-', '\-', regex=True)

In [72]:
georeport= final_reports[[
    'case_number', 
    'status', 
    'opened_date', 
    'last_modified_date', 
    'closed_date',
    'status',
    'case_origin',
    'problem_category',
    'problem_category_detail',
    'subject',
    'problem_group',
    'address',
    'lat',
    'lng']]
georeport = georeport.reset_index(drop=True)
georeport.head()

,case_number,status,opened_date,last_modified_date,closed_date,status,case_origin,problem_category,problem_category_detail,subject,problem_group,address,lat,lng
0,00127913,New,2016-12-28 11:44:00,2016-12-28 11:44:00,NaT,New,Mobile,Graffiti,,Graffiti at 1285-1287 Lincoln Avenue San Dieg...,Other,"1285-1287 Lincoln Avenue San Diego, CA 92103",32.752064000000000,-117.151381000000000
1,00127912,New,2016-12-28 11:44:00,2016-12-28 11:44:00,NaT,New,Mobile,Graffiti,,"Graffiti at 4117 41st Street San Diego, CA 92105",Other,"4117 41st Street San Diego, CA 92105",32.751802000000000,-117.106234000000000
2,00127910,Acknowledged,2016-12-28 11:40:00,2016-12-28 11:43:00,NaT,Acknowledged,Phone,Pothole,,"Pothole at 2900 PENNANT WAY, 92122",Streets,"2900 PENNANT WAY, 92122",32.847394508819896,-117.222866552706400
3,00127909,New,2016-12-28 11:40:00,2016-12-28 11:40:00,NaT,New,Mobile,Other,,"Other at 5298 Cromwell Court San Diego, CA 92116",Other,"5298 Cromwell Court San Diego, CA 92116",32.775158000000000,-117.118126000000000
4,00127908,Acknowledged,2016-12-28 11:35:00,2016-12-28 11:36:00,NaT,Acknowledged,Phone,Street Sweeping,Street Sweeping,"Debris in Street at 6300 RIVERDALE ST, 92120",Streets,"6300 RIVERDALE ST, 92120",32.788772881678380,-117.099617471504290


In [74]:
georeport = georeport.rename(columns={
    'case_number': 'service_request_id',
    'opened_date': 'requested_datetime',
    'last_modified_date': 'updated_datetime',
    'closed_date': 'closed_date',
    'status': 'status_description', 
    'case_origin': 'source',
    'problem_category': 'service_name',
    'problem_category_detail': 'service_subtype',
    'subject': 'description',
    'problem_group': 'agency_responsible',
    'address': 'address',
    'lat': 'lat',
    'lng': 'long'
})

georeport.head()

,service_request_id,status_description,requested_datetime,updated_datetime,closed_date,status_description,source,service_name,service_subtype,description,agency_responsible,address,lat,long
0,00127913,New,2016-12-28 11:44:00,2016-12-28 11:44:00,NaT,New,Mobile,Graffiti,,Graffiti at 1285-1287 Lincoln Avenue San Dieg...,Other,"1285-1287 Lincoln Avenue San Diego, CA 92103",32.752064000000000,-117.151381000000000
1,00127912,New,2016-12-28 11:44:00,2016-12-28 11:44:00,NaT,New,Mobile,Graffiti,,"Graffiti at 4117 41st Street San Diego, CA 92105",Other,"4117 41st Street San Diego, CA 92105",32.751802000000000,-117.106234000000000
2,00127910,Acknowledged,2016-12-28 11:40:00,2016-12-28 11:43:00,NaT,Acknowledged,Phone,Pothole,,"Pothole at 2900 PENNANT WAY, 92122",Streets,"2900 PENNANT WAY, 92122",32.847394508819896,-117.222866552706400
3,00127909,New,2016-12-28 11:40:00,2016-12-28 11:40:00,NaT,New,Mobile,Other,,"Other at 5298 Cromwell Court San Diego, CA 92116",Other,"5298 Cromwell Court San Diego, CA 92116",32.775158000000000,-117.118126000000000
4,00127908,Acknowledged,2016-12-28 11:35:00,2016-12-28 11:36:00,NaT,Acknowledged,Phone,Street Sweeping,Street Sweeping,"Debris in Street at 6300 RIVERDALE ST, 92120",Streets,"6300 RIVERDALE ST, 92120",32.788772881678380,-117.099617471504290
